# EntropyLab Quickstart ⚡
Backtest SPY in under a minute. If `entropylab` is installed, we’ll use it; otherwise we’ll run a tiny fallback to demonstrate the flow.

In [ ]:
import sys, subprocess, importlib
def pip_install(pkg):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

try:
    import yfinance as yf
except Exception:
    pip_install('yfinance>=0.2.40')
    import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

start = '2020-01-01'
end = '2020-12-31'
spy = yf.download('SPY', start=start, end=end, auto_adjust=True)
prices = spy['Close'].dropna()
prices.head()

In [ ]:
def fallback_backtest(prices: pd.Series, risk_free=0.0):
    rets = prices.pct_change().dropna()
    if rets.std() == 0:
        sharpe = 0.0
    else:
        sharpe = ((rets.mean() - risk_free/252) / rets.std()) * np.sqrt(252)
    nav = (1 + rets).cumprod()
    print(f"Sharpe (fallback): {sharpe:.2f}")
    return {
        'sharpe': float(sharpe),
        'final_return': float(nav.iloc[-1] - 1)
    }

try:
    entropylab = importlib.import_module('entropylab')
    if hasattr(entropylab, 'backtest'):
        result = entropylab.backtest(prices)  # expected to print/return Sharpe
        print('Sharpe (entropylab):', getattr(result, 'sharpe', result))
    else:
        result = fallback_backtest(prices)
except Exception as e:
    print('[Info] entropylab not found or backtest() missing, using fallback.')
    result = fallback_backtest(prices)
result

### 🚀 Want to plug this into Interactive Brokers automatically?  
**Upgrade to EntropyLab Pro ($299 one-time)**